In [ ]:
!pwd

In [ ]:
import wandb

Add your wand-b key to kaggle secrets or mention them in code

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wand-key")

In [ ]:
wandb.login(key=secret_value_0)

In [ ]:
!git clone https://github.com/devanshigarg01/pittsburghdata.git

In [ ]:
!git clone https://github.com/amaralibey/MixVPR.git


In [ ]:
!pip install -r'./MixVPR/requirements.txt'

In [ ]:
!git clone https://github.com/amaralibey/gsv-cities.git

In [ ]:
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118

In [ ]:
ls

In [ ]:
cd MixVPR

In [2]:
import pandas as pd
import os
import numpy as np

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import pairwise_distances
import numpy as np
from torchvision import transforms as T
import pytorch_lightning as pl
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from torch.optim import lr_scheduler, optimizer
import utils
from models import helper
from pytorch_lightning.loggers import WandbLogger

In [ ]:
pwd

In [ ]:
with open("main.py", "r") as file:
    text = file.read()

Uncomment and use the scheduler which is required. To use the original scheduler, skip this step.

In [ ]:
old = "scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=self.milestones, gamma=self.lr_mult)"
new = "scheduler = lr_scheduler.CyclicLR(optimizer,base_lr=0.001,max_lr=0.1,mode='triangular2')"
#new = "scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)"
#new = "scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.5)"

In [ ]:
text = text.replace(old,new)

In [ ]:
with open("main.py", "w") as file:
    file.write(text)

In [ ]:
cd dataloaders

In [ ]:
with open("GSVCitiesDataset.py", "r") as file:
    code = file.read()

Change the path to where the gsv cities dataset is stored


In [ ]:
BASE_PATH = '../datasets/gsv_cities/'
BASE_PATH_new = '/kaggle/input/gsv-cities/'

In [ ]:
code = code.replace(BASE_PATH,BASE_PATH_new)

In [ ]:
with open("GSVCitiesDataset.py", "w") as file:
    file.write(code)

In [ ]:
with open("PittsburgDataset.py", "r") as file:
    code2 = file.read()

Change the path to where the pittsburg dataset is cloned

In [ ]:
root_dir = '../datasets/Pittsburgh/'
root_dir_new = "/kaggle/working/pittsburghdata/"

In [ ]:
code2 = code2.replace(root_dir,root_dir_new)

In [ ]:
with open("PittsburgDataset.py", "w") as file:
    file.write(code2)

In [ ]:
with open("MapillaryDataset.py", "r") as file:
    code3 = file.read()

Change the path to where the mapillary dataset is stored

In [ ]:
DATASET_ROOT = '../datasets/msls_val/'
DATASET_ROOT_new = '/kaggle/working/MixVPR/datasets/msls_val/'

In [ ]:
code3 = code3.replace(DATASET_ROOT,DATASET_ROOT_new)

In [ ]:
with open("MapillaryDataset.py", "w") as file:
    file.write(code3)

In [ ]:
cd ..

In [3]:
root = '/kaggle/input/gsv-cities/Dataframes/'
cities = os.listdir('/kaggle/input/gsv-cities/Dataframes/')
mp = dict()

for city in cities:
    path = root + city
    df = pd.read_csv(path)
    images = len(df)
    mp[city] = images

In [4]:
mp

{'Lisbon.csv': 27045,
 'Rome.csv': 24068,
 'Minneapolis.csv': 22326,
 'LosAngeles.csv': 8891,
 'Medellin.csv': 6024,
 'BuenosAires.csv': 8481,
 'Madrid.csv': 14554,
 'Miami.csv': 43637,
 'Bangkok.csv': 22271,
 'Boston.csv': 32616,
 'Chicago.csv': 34091,
 'OSL.csv': 9756,
 'TRT.csv': 17712,
 'PRG.csv': 17590,
 'WashingtonDC.csv': 11545,
 'Barcelona.csv': 15894,
 'Melbourne.csv': 28542,
 'Phoenix.csv': 36251,
 'Brussels.csv': 14171,
 'Osaka.csv': 22605,
 'London.csv': 58672,
 'PRS.csv': 39963,
 'MexicoCity.csv': 12801}

In [ ]:
from main import VPRModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
cd dataloaders

In [ ]:
from dataloaders.GSVCitiesDataset import GSVCitiesDataset
from dataloaders import PittsburgDataset
#from dataloaders import MapillaryDataset

change the dataset_root path here as well

In [ ]:
from pathlib import Path
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

DATASET_ROOT = '/kaggle/input/msls-val/'

path_obj = Path(DATASET_ROOT)
if not path_obj.exists():
    raise Exception('Please make sure the path to mapillary_sls dataset is correct')

if not path_obj.joinpath('train_val'):
    raise Exception(f'Please make sure the directory train_val from mapillary_sls dataset is situated in the directory {DATASET_ROOT}')

class MSLS(Dataset):
    def __init__(self, input_transform = None):
        
        self.input_transform = input_transform
        
        # hard coded reference image names, this avoids the hassle of listing them at each epoch.
        self.dbImages = np.load('../datasets/msls_val/msls_val_dbImages.npy')
        
        # hard coded query image names.
        self.qImages = np.load('../datasets/msls_val/msls_val_qImages.npy')
        
        # hard coded index of query images
        self.qIdx = np.load('../datasets/msls_val/msls_val_qIdx.npy')
        
        # hard coded groundtruth (correspondence between each query and its matches)
        self.pIdx = np.load('../datasets/msls_val/msls_val_pIdx.npy', allow_pickle=True)
        
        # concatenate reference images then query images so that we can use only one dataloader
        self.images = np.concatenate((self.dbImages, self.qImages[self.qIdx]))
        
        # we need to keeo the number of references so that we can split references-queries 
        # when calculating recall@K
        self.num_references = len(self.dbImages)
    
    def __getitem__(self, index):
        image_path = DATASET_ROOT + self.images[index]

        if not os.path.exists(image_path):
            # Image doesn't exist, return an empty placeholder or take any other action you prefer
            return None

        img = Image.open(image_path)

        if self.input_transform:
            img = self.input_transform(img)

        return img, index
        

    def __len__(self):
        return len(self.images)

In [ ]:
from prettytable import PrettyTable

IMAGENET_MEAN_STD = {'mean': [0.485, 0.456, 0.406], 
                     'std': [0.229, 0.224, 0.225]}

VIT_MEAN_STD = {'mean': [0.5, 0.5, 0.5], 
                'std': [0.5, 0.5, 0.5]}

Add the cities which you want to train on here from this list:
"'Bangkok',
    'BuenosAires',
    'LosAngeles',
    'MexicoCity',
    'OSL',
    'Rome',
    'Barcelona',
    'Chicago',
    'Madrid',
    'Miami',
    'Phoenix',
    'TRT',
    'Boston',
    'Lisbon',
    'Medellin',
    'Minneapolis',
    'PRG',
    'WashingtonDC',
    'Brussels',
    'London',
    'Melbourne',
    'Osaka',
    'PRS',"

In [ ]:
TRAIN_CITIES = [
    'Medellin', 
    'LosAngeles',
    'WashingtonDC',  
]

In [ ]:
class GSVCitiesDataModule(pl.LightningDataModule):
    def __init__(self,
                 batch_size=32,
                 img_per_place=4,
                 min_img_per_place=4,
                 shuffle_all=False,
                 image_size=(480, 640),
                 num_workers=4,
                 show_data_stats=True,
                 cities=TRAIN_CITIES,
                 mean_std=IMAGENET_MEAN_STD,
                 batch_sampler=None,
                 random_sample_from_each_place=True,
                 val_set_names=['pitts30k_val', 'msls_val']
                 ):
        super().__init__()
        self.batch_size = batch_size
        self.img_per_place = img_per_place
        self.min_img_per_place = min_img_per_place
        self.shuffle_all = shuffle_all
        self.image_size = image_size
        self.num_workers = num_workers
        self.batch_sampler = batch_sampler
        self.show_data_stats = show_data_stats
        self.cities = cities
        self.mean_dataset = mean_std['mean']
        self.std_dataset = mean_std['std']
        self.random_sample_from_each_place = random_sample_from_each_place
        self.val_set_names = val_set_names
        self.save_hyperparameters() # save hyperparameter with Pytorch Lightening

        self.train_transform = T.Compose([
            T.Resize(image_size, interpolation=T.InterpolationMode.BILINEAR),
            T.RandAugment(num_ops=3, interpolation=T.InterpolationMode.BILINEAR),
            T.ToTensor(),
            T.Normalize(mean=self.mean_dataset, std=self.std_dataset),
        ])

        self.valid_transform = T.Compose([
            T.Resize(image_size, interpolation=T.InterpolationMode.BILINEAR),
            T.ToTensor(),
            T.Normalize(mean=self.mean_dataset, std=self.std_dataset)])

        self.train_loader_config = {
            'batch_size': self.batch_size,
            'num_workers': self.num_workers,
            'drop_last': False,
            'pin_memory': True,
            'shuffle': self.shuffle_all}

        self.valid_loader_config = {
            'batch_size': self.batch_size,
            'num_workers': self.num_workers//2,
            'drop_last': False,
            'pin_memory': True,
            'shuffle': False}

    def setup(self, stage):
        if stage == 'fit':
            # load train dataloader with reload routine
            self.reload()

            # load validation sets (pitts_val, msls_val, ...etc)
            self.val_datasets = []
            for valid_set_name in self.val_set_names:
                if valid_set_name.lower() == 'pitts30k_test':
                    self.val_datasets.append(PittsburgDataset.get_whole_test_set(
                        input_transform=self.valid_transform))
                elif valid_set_name.lower() == 'pitts30k_val':
                    self.val_datasets.append(PittsburgDataset.get_whole_val_set(
                        input_transform=self.valid_transform))
                elif valid_set_name.lower() == 'msls_val':
                    self.val_datasets.append(MSLS(
                        input_transform=self.valid_transform))
                else:
                    print(
                        f'Validation set {valid_set_name} does not exist or has not been implemented yet')
                    raise NotImplementedError
            if self.show_data_stats:
                self.print_stats()

    def reload(self):
        self.train_dataset = GSVCitiesDataset(
            cities=self.cities,
            img_per_place=self.img_per_place,
            min_img_per_place=self.min_img_per_place,
            random_sample_from_each_place=self.random_sample_from_each_place,
            transform=self.train_transform)

    def train_dataloader(self):
        self.reload()
        return DataLoader(dataset=self.train_dataset, **self.train_loader_config)

    def val_dataloader(self):
        val_dataloaders = []
        for val_dataset in self.val_datasets:
            val_dataloaders.append(DataLoader(
                dataset=val_dataset, **self.valid_loader_config))
        return val_dataloaders

    def print_stats(self):
        print()  # print a new line
        table = PrettyTable()
        table.field_names = ['Data', 'Value']
        table.align['Data'] = "l"
        table.align['Value'] = "l"
        table.header = False
        table.add_row(["# of cities", f"{len(TRAIN_CITIES)}"])
        table.add_row(["# of places", f'{self.train_dataset.__len__()}'])
        table.add_row(["# of images", f'{self.train_dataset.total_nb_images}'])
        print(table.get_string(title="Training Dataset"))
        print()

        table = PrettyTable()
        table.field_names = ['Data', 'Value']
        table.align['Data'] = "l"
        table.align['Value'] = "l"
        table.header = False
        for i, val_set_name in enumerate(self.val_set_names):
            table.add_row([f"Validation set {i+1}", f"{val_set_name}"])
        # table.add_row(["# of places", f'{self.train_dataset.__len__()}'])
        print(table.get_string(title="Validation Datasets"))
        print()

        table = PrettyTable()
        table.field_names = ['Data', 'Value']
        table.align['Data'] = "l"
        table.align['Value'] = "l"
        table.header = False
        table.add_row(
            ["Batch size (PxK)", f"{self.batch_size}x{self.img_per_place}"])
        table.add_row(
            ["# of iterations", f"{self.train_dataset.__len__()//self.batch_size}"])
        table.add_row(["Image size", f"{self.image_size}"])
        print(table.get_string(title="Training config"))

In [ ]:
datamodule = GSVCitiesDataModule(
        batch_size=8,
        img_per_place=4,
        min_img_per_place=4,
        shuffle_all=False, # shuffle all images or keep shuffling in-city only
        random_sample_from_each_place=True,
        image_size=(320, 320),
        num_workers=4,
        show_data_stats=True,
        val_set_names=['msls_val'], # pitts30k_val, pitts30k_test, msls_val
    )

In [ ]:
model = VPRModel(
        #---- Encoder
        backbone_arch='resnet50',
        pretrained=True,
        layers_to_freeze=2,
        layers_to_crop=[4], # 4 crops the last resnet layer, 3 crops the 3rd, ...etc
        agg_arch='MixVPR',
        agg_config={'in_channels' : 1024,
                'in_h' : 20,
                'in_w' : 20,
                'out_channels' : 1024,
                'mix_depth' : 4,
                'mlp_ratio' : 1,
                'out_rows' : 4}, # the output dim will be (out_rows * out_channels)
        
        #---- Train hyperparameters
        lr=0.05, # 0.0002 for adam, 0.05 or sgd (needs to change according to batch size)
        optimizer='sgd', # sgd, adamw
        weight_decay=0.001, # 0.001 for sgd and 0 for adam,
        momentum=0.9,
        warmpup_steps=650,
        milestones=[5, 10, 15, 25, 45],
        lr_mult=0.3,
        loss_name='MultiSimilarityLoss',
        miner_name='MultiSimilarityMiner', # example: TripletMarginMiner, MultiSimilarityMiner, PairMarginMiner
        miner_margin=0.1,
        faiss_gpu=False
    )

In [ ]:
checkpoint_cb = ModelCheckpoint(
        monitor='msls_val/R1',
        filename=f'{model.encoder_arch}' +
        '_epoch({epoch:02d})_step({step:04d})_R1[{msls_val/R1:.4f}]_R5[{msls_val/R5:.4f}]',
        auto_insert_metric_name=False,
        save_weights_only=True,
        save_top_k=3,
        mode='max',)

change the run name 

In [ ]:
wandb.init(project="mix-vpr", name="cyclic-10-epoch-3-city")

In [ ]:
wandb_logger = WandbLogger()

number of epochs can be changed here

In [ ]:
trainer = pl.Trainer(
        accelerator='gpu', devices=[0],
        default_root_dir=f'./LOGS/{model.encoder_arch}', # Tensorflow can be used to viz 

        num_sanity_val_steps=0, # runs a validation step before stating training
        precision=16, # we use half precision to reduce  memory usage
        max_epochs=10,
        check_val_every_n_epoch=1, # run validation every epoch
        callbacks=[checkpoint_cb],# we only run the checkpointing callback (you can add more)
        reload_dataloaders_every_n_epochs=1, # we reload the dataset to shuffle the order
        log_every_n_steps=20,
        logger=wandb_logger,
        # fast_dev_run=True # uncomment or dev mode (only runs a one iteration train and validation, no checkpointing).
    )

In [ ]:
trainer.fit(model=model, datamodule=datamodule)